# BÀI TẬP SQL CHƯƠNG 3  
# 37_LUONGDUCTHANG  

# Câu 1: Hãy viết câu lệnh SQL để tính sự tương quan giữa A và B theo công thức

In [ ]:
import sqlite3
import math

# Kết nối SQLite
conn = sqlite3.connect("test.db")
cursor = conn.cursor()

# Đăng ký các hàm toán học vào SQLite
conn.create_function("SQRT", 1, math.sqrt)
conn.create_function("POWER", 2, math.pow)

# Tạo bảng và thêm dữ liệu
cursor.executescript('''
DROP TABLE IF EXISTS sample_data;

CREATE TABLE sample_data (
    id INTEGER PRIMARY KEY,
    column_A FLOAT,
    column_B FLOAT
);

INSERT INTO sample_data (id, column_A, column_B) VALUES
(1, 10, 8),
(2, 9, 7),
(3, 8, 6),
(4, 7, 5),
(5, 6, 4);
''')

# Tính Pearson correlation trong SQL
cursor.execute('''
SELECT 
  'column_A vs column_B' AS pair,
  (
    COUNT(column_A) * SUM(column_A * column_B) - SUM(column_A) * SUM(column_B)
  ) / (
    SQRT(COUNT(column_A) * SUM(POWER(column_A, 2)) - POWER(SUM(column_A), 2)) *
    SQRT(COUNT(column_B) * SUM(POWER(column_B, 2)) - POWER(SUM(column_B), 2))
  ) AS corr_AB
FROM sample_data;
''')

result = cursor.fetchone()
print(f"{result[0]}: {result[1]:.4f}")


column_A vs column_B: 1.0000


# Câu2

In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("car.db")
cursor = conn.cursor()

In [93]:
cursor.execute("Drop table if exists car_agency")

cursor.execute(
    '''create table car_agency(
    days TEXT primary key,
    A float not null,
    B float not null,
    C float not null
    )'''
)

conn.commit()

In [94]:
cursor.executescript('''
INSERT INTO car_agency (days,A,B,C) values
('day1', 8,9,7),
('day2', 7.5,8.5,7),
('day3', 6,7,8),
('day4', 7,6,5);
''')

conn.commit

<function Connection.commit()>

In [95]:
cursor.execute(" select * from car_agency")

rows = cursor.fetchall()
for row in rows:
    print(row)

('day1', 8.0, 9.0, 7.0)
('day2', 7.5, 8.5, 7.0)
('day3', 6.0, 7.0, 8.0)
('day4', 7.0, 6.0, 5.0)


chuyển dạng bảng ban đầu sang dạng quan hệ

In [96]:
cursor.execute("drop table if exists car_score")

cursor.execute('''
                    create table car_score(
                    day TEXT,
                    car_model TEXT,
                    score REAL)

''')

conn.commit()

In [97]:
cursor.executescript('''
INSERT INTO car_score (day, car_model, score) VALUES
('day 1', 'A', 8),
('Day 1', 'B', 9),
('Day 1', 'C', 7),
('Day 2', 'A', 7.5),
('Day 2', 'B', 8.5),
('Day 2', 'C', 7),
('Day 3', 'A', 6),
('Day 3', 'B', 7),
('Day 3', 'C', 8),
('Day 4', 'A', 7),
('Day 4', 'B', 6),
('Day 4', 'C', 5);
''')

conn.commit()

In [98]:
cursor.execute("select * from car_score")

rows = cursor.fetchall()
for row in rows:
    print(row)

('day 1', 'A', 8.0)
('Day 1', 'B', 9.0)
('Day 1', 'C', 7.0)
('Day 2', 'A', 7.5)
('Day 2', 'B', 8.5)
('Day 2', 'C', 7.0)
('Day 3', 'A', 6.0)
('Day 3', 'B', 7.0)
('Day 3', 'C', 8.0)
('Day 4', 'A', 7.0)
('Day 4', 'B', 6.0)
('Day 4', 'C', 5.0)


Kiểm định χ² (Chi-squared test) dùng cho dữ liệu phân loại (categorical), không phải dữ liệu liên tục như điểm số nên ta phải nhóm chúng lại trước khi thực hiện    

score được nhóm như sau:       
[0-5): kém  
[5-7): trung bình   
[7-9): tốt  
[9-10]: rất tốt 

phân loại điểm số

In [99]:
cursor.execute("DROP VIEW IF EXISTS car_scores_classifieds")

cursor.execute('''
CREATE VIEW car_scores_classifieds AS
SELECT
    day,
    car_model,
    score,
    CASE
        WHEN score < 5 THEN 'Kém'
        WHEN score >= 5 AND score < 7 THEN 'Trung bình'
        WHEN score >= 7 AND score < 9 THEN 'Tốt'
        ELSE 'Rất tốt'
    END AS category
FROM car_score;
''')

conn.commit()


đếm tần suất cho từng loại điểm theo từng mẫu

In [100]:
cursor.execute('''
SELECT car_model, category, COUNT(*) as count
FROM car_scores_classifieds
GROUP BY car_model, category
ORDER BY car_model, category;

''')

conn.commit()

rows = cursor.fetchall()
for row in rows:
    print(row)

('A', 'Trung bình', 1)
('A', 'Tốt', 3)
('B', 'Rất tốt', 1)
('B', 'Trung bình', 1)
('B', 'Tốt', 2)
('C', 'Trung bình', 1)
('C', 'Tốt', 3)


kiểm điểm X^2

In [101]:
import pandas as pd

df = pd.read_sql_query('''
SELECT car_model, category, COUNT(*) as count
FROM car_scores_classifieds
GROUP BY car_model, category
''', conn)

print(df)


  car_model    category  count
0         A  Trung bình      1
1         A         Tốt      3
2         B     Rất tốt      1
3         B  Trung bình      1
4         B         Tốt      2
5         C  Trung bình      1
6         C         Tốt      3


In [102]:
from scipy.stats import chi2_contingency

# Pivot để có ma trận tần suất
table = df.pivot(index='car_model', columns='category', values='count').fillna(0)
print(table)

chi2, p, dof, expected = chi2_contingency(table)
print("Chi-squared:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)


category   Rất tốt  Trung bình  Tốt
car_model                          
A              0.0         1.0  3.0
B              1.0         1.0  2.0
C              0.0         1.0  3.0
Chi-squared: 2.25
p-value: 0.6898864931364932
Degrees of freedom: 4


- Kết quả kiểm định Chi-squared:
Giá trị Chi-squared = 2.25

p-value = 0.6899

Degrees of freedom = 4

Vì p-value lớn hơn 0.05, ta không có đủ bằng chứng thống kê để bác bỏ giả thuyết không (H₀), tức là:

Không có sự khác biệt đáng kể về điểm số giữa các mẫu xe.

- Kết luận:
Không có mẫu xe nào nổi bật một cách đáng kể.

Phân phối điểm số không phụ thuộc vào loại xe.

câu 3

In [106]:
import sqlite3

# Kết nối SQLite
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Tạo bảng flights và thêm dữ liệu mẫu
cursor.executescript("""
DROP TABLE IF EXISTS flights;

CREATE TABLE flights (
    id INTEGER PRIMARY KEY,
    departure_time INTEGER
);

INSERT INTO flights (departure_time) VALUES
(830),
(1445),
(0),
(1130),
(2359);
""")

cursor.execute("""
SELECT
  departure_time,
  printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights;
""")

# Hiển thị kết quả
rows = cursor.fetchall()
for row in rows:
    print(f"Raw: {row[0]} -> Formatted: {row[1]}")


Raw: 830 -> Formatted: 08:30
Raw: 1445 -> Formatted: 14:45
Raw: 0 -> Formatted: 00:00
Raw: 1130 -> Formatted: 11:30
Raw: 2359 -> Formatted: 23:59


câu 4

In [107]:
import sqlite3
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

cursor.executescript('''
DROP TABLE IF EXISTS sample_data;

CREATE TABLE sample_data (
    id INTEGER PRIMARY KEY,
    value REAL
);

INSERT INTO sample_data (value) VALUES
(10), (12), (13), (12), (12), (11), (14), (100); -- 100 là giá trị ngoại lệ
''')


Tính toán giá trị trung vị (median)

In [108]:
# Lấy danh sách giá trị và tính median trong Python
cursor.execute("SELECT value FROM sample_data ORDER BY value")
values = [row[0] for row in cursor.fetchall()]
n = len(values)

if n % 2 == 1:
    median = values[n // 2]
else:
    median = (values[n // 2 - 1] + values[n // 2]) / 2

print(f"Median: {median}")


Median: 12.0


Tính MAD và phát hiện ngoại lệ

In [109]:
# Tạo bảng tạm chứa độ lệch tuyệt đối
cursor.execute("DROP TABLE IF EXISTS abs_deviation")
cursor.execute("CREATE TABLE abs_deviation AS SELECT ABS(value - ?) AS deviation FROM sample_data", (median,))

# Lấy MAD (trung vị của deviation)
cursor.execute("SELECT deviation FROM abs_deviation ORDER BY deviation")
devs = [row[0] for row in cursor.fetchall()]
n_dev = len(devs)

if n_dev % 2 == 1:
    mad = devs[n_dev // 2]
else:
    mad = (devs[n_dev // 2 - 1] + devs[n_dev // 2]) / 2

print(f"MAD: {mad}")


MAD: 1.0


Truy vấn phát hiện ngoại lệ

In [ ]:
threshold = 1.5 * mad

# |value - 12| > 1.5 * 1.0 = 1.5
cursor.execute("SELECT id, value FROM sample_data WHERE ABS(value - ?) > ?", (median, threshold))
outliers = cursor.fetchall()

print("Các ngoại lệ (outliers):")
for row in outliers:
    print(row)


Các ngoại lệ (outliers):
(1, 10.0)
(7, 14.0)
(8, 100.0)


Nhận xét:
Các điểm như 10, 14 lệch tương đối nhẹ nhưng vẫn vượt ngưỡng → ngoại lệ nhẹ.

100 là một ngoại lệ rất mạnh, rõ ràng khác biệt với phần còn lại → có thể là lỗi hoặc dữ liệu đáng kiểm tra lại.

In [112]:
import sqlite3

# Kết nối SQLite
conn = sqlite3.connect("patient.db")  # Dùng bộ nhớ tạm
cursor = conn.cursor()

# 1. Tạo bảng Patient và chèn dữ liệu mẫu
cursor.executescript("""
DROP TABLE IF EXISTS Patient;

CREATE TABLE Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight REAL,
    height REAL
);

INSERT INTO Patient (id, last_name, weight, height) VALUES
(1, 'Nguyen', 60, 165),
(2, 'Nguyen', 61, 165),
(3, 'Le',     80, 170),
(4, 'Nguyen', 95, 180);
""")

# 2. Truy vấn xác định khả năng trùng
cursor.execute("""
SELECT 
    p1.id AS id1,
    p2.id AS id2,
    p1.last_name,
    p2.last_name,
    p1.weight,
    p2.weight,
    CASE WHEN p1.last_name = p2.last_name THEN 1 ELSE 0 END AS last_name_match,
    CASE WHEN ABS(p1.weight - p2.weight) <= 2 THEN 1 ELSE 0 END AS weight_match,
    CASE 
        WHEN (p1.last_name = p2.last_name) AND (ABS(p1.weight - p2.weight) <= 2) 
        THEN 'Có thể là cùng người'
        ELSE 'Khác người'
    END AS match_result
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id;
""")

# 3. In kết quả
rows = cursor.fetchall()
for row in rows:
    print(row)

# Đóng kết nối
conn.close()


(1, 2, 'Nguyen', 'Nguyen', 60.0, 61.0, 1, 1, 'Có thể là cùng người')
(1, 3, 'Nguyen', 'Le', 60.0, 80.0, 0, 0, 'Khác người')
(1, 4, 'Nguyen', 'Nguyen', 60.0, 95.0, 1, 0, 'Khác người')
(2, 3, 'Nguyen', 'Le', 61.0, 80.0, 0, 0, 'Khác người')
(2, 4, 'Nguyen', 'Nguyen', 61.0, 95.0, 1, 0, 'Khác người')
(3, 4, 'Le', 'Nguyen', 80.0, 95.0, 0, 0, 'Khác người')
